In [10]:
import torch
import cv2
import csv
import json
import numpy as np
import pandas as pd
# from object_detection import ObjectDetection
import math
import subprocess
from tqdm import tqdm
# import maths

In [11]:
# Initialize YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)
model.eval()

Using cache found in C:\Users\bedo-/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-4-18 Python-3.9.16 torch-2.0.0 CUDA:0 (NVIDIA GeForce GTX 1660, 6144MiB)



requirements: C:\Users\bedo-\.cache\torch\hub\requirements.txt not found, check failed.


Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 48, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1))
  

In [80]:
def track_objects(video_path, output_path, frame_cut:int, json_filename):
    """Takes video input
    frame_cut: upper limit for the obj detector in the given video """
    
    cap = cv2.VideoCapture(video_path)
     #Get the video frame rate and dimensions
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width =  int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height =  int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    ffmpeg_cmd = f"ffmpeg -y -f rawvideo -pix_fmt bgr24 -s {frame_width}x{frame_height} -r {fps} -i - -c:v libx264 -preset fast -crf 30 -pix_fmt nv12 -an -vcodec libx264 {output_path}"

    # Open the output file for writing 
    output_file = subprocess.Popen(ffmpeg_cmd.split(' '), stdin=subprocess.PIPE)
    
    # Define the loading animation
    pbar = tqdm(total= cap.get(cv2.CAP_PROP_FRAME_COUNT), desc="Processing video", unit="frames" )
    
    # Initialize count and bboxes points
    count = 0
    bboxes_prev_frame = []

    tracking_objects = {}
    track_id = 0

    object_dicts = {}
    
    while True:
        ret, frame = cap.read() 
        count += 1

        if not ret:
            break      

        # Point current frame
        bboxes_cur_frame = []

        # Detect objects on frame using YOLOv5
        frame_model = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model(frame_model[frame_cut:])
        for box in results.xyxy[0].tolist():
            (x1, y1, x2, y2, conf, class_ids) = box
            if conf<0.6: continue
            x1, y1, x2, y2 =  x1,  y1 + frame_cut,  x2,  y2+frame_cut
            cx =  (x1 + x2 ) / 2
            cy =  (y1 + y2 ) / 2
            bboxes_cur_frame.append([x1,y1,x2,y2])

            # Draw bounding box around detected object
            cv2.rectangle(frame, int(x1), int(y1), int(x2), int(y2), (255, 174, 66), 2)
        
        if count <= 2:
            for pt in bboxes_cur_frame:
                x1,y1,x2,y2 = pt
                cx1 =  (x1 + x2 ) / 2
                cy1 =  (y1 + y2 ) / 2
                diagonal = ( ( (x2-x1)**2) + ( (y2-y1)**2) ) ** 0.5
                for x1,y1,x2,y2 in bboxes_prev_frame:
                    cx2 =  (x1 + x2 ) / 2
                    cy2 =  (y1 + y2 ) / 2
                    distance = ( ( (cx2-cx1)**2) + ( (cy2-cy1)**2) ) ** 0.5

                    if distance < 20:
                        print( f'loop one, id:{track_id}, dis {distance}, diag {diagonal}' )
                        tracking_objects[track_id] = pt
                        track_id += 1
                        break
        else:

            tracking_objects_copy = tracking_objects.copy()
            bboxes_cur_frame_copy = bboxes_cur_frame.copy()

            for object_id, pt2 in tracking_objects_copy.items():
                object_exists = False
                x1,y1,x2,y2 = pt2
                cx2 =  (x1 + x2 ) / 2
                cy2 =  (y1 + y2 ) / 2
                for pt in bboxes_cur_frame_copy:
                    x1,y1,x2,y2 = pt
                    cx1 =  (x1 + x2 ) / 2
                    cy1 =  (y1 + y2 ) / 2
                    diagonal = ( ( (x2-x1)**2) + ( (y2-y1)**2) ) ** 0.5
                    distance = ( ( (cx2-cx1)**2) + ( (cy2-cy1)**2) ) ** 0.5

                    # Update IDs position
                    if distance < 20:
                        print( f'loop one, id:{track_id}, dis {distance}, diag {diagonal}' )
                        tracking_objects[object_id] = pt
                        object_exists = True
                        if pt in bboxes_cur_frame:
                            bboxes_cur_frame.remove(pt)
                        break

                # Remove IDs lost
                if not object_exists:
                    tracking_objects.pop(object_id)

            # Add new IDs found
            for pt in bboxes_cur_frame:
                tracking_objects[track_id] = pt
                track_id += 1
        
        for object_id, pt in tracking_objects.items():
#             if object_id != 0: continue
#             # Create a dictionary for the object if it does not exist
#             if object_id not in object_dicts:
#                 object_dicts[object_id] = []
                
#             object_dicts[object_id].append({
#                 'frame': count,
#                 'x1': x1,
#                 'y1': y1,
#                 'x2': x2,
#                 'y2': y2  
#             })
            cx1 =  (x1 + x2 ) / 2
            cy1 =  (y1 + y2 ) / 2
            cv2.circle(frame, (cx1,cy1), 5, (160, 32, 140), -1)
            cv2.putText(frame, str(object_id), (cx1, cy1 - 7), 0, 1, (255, 255, 255), 2)

        bboxes_prev_frame = bboxes_cur_frame.copy()
        # Update the loading animation
        pbar.update(1)
        
        # Write the output frame to the output file
        output_file.stdin.write(frame.tobytes())  

    # Release the video capture and close the output file
    cap.release()
    output_file.stdin.close()
    output_file.wait()
#     cv2.destroyAllWindows()
    pbar.close()
        
    # Write object dictionaries to JSON file
    with open(json_filename, 'w') as f:
        json.dump(object_dicts, f, indent=4)


In [81]:
track_objects('light traffic.mp4','track_test.mp4',200,'tracked_objects.json')





Processing video:   0%|                                                                  | 0/335.0 [00:00<?, ?frames/s]



Processing video:   0%|▏                                                         | 1/335.0 [00:00<01:14,  4.48frames/s]



Processing video:   1%|▋                                                         | 4/335.0 [00:00<00:23, 14.39frames/s]



Processing video:   2%|█▍                                                        | 8/335.0 [00:00<00:15, 21.47frames/s]



Processing video:   4%|██                                                       | 12/335.0 [00:00<00:12, 25.95frames/s]



Processing video:   5%|██▋                                                      | 16/335.0 [00:00<00:11, 28.79frames/s]



Processing video:   6%|███▍                                                     | 20/335.0 [00:00<00:10, 30.63frames/s]

error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - Can't parse 'pt1'. Input argument doesn't provide sequence protocol
>  - Can't parse 'pt1'. Input argument doesn't provide sequence protocol
>  - rectangle() takes at most 6 arguments (7 given)
>  - rectangle() takes at most 6 arguments (7 given)






Processing video:   7%|███▋                                                     | 22/335.0 [00:18<00:10, 30.63frames/s]

In [52]:
# Read object dictionaries from JSON file
with open('tracked_objects.json', 'r') as f:
    object_dicts = json.load(f)

# Print object dictionaries in a readable way
print(json.dumps(object_dicts, indent=4))


{}


In [1]:
t = [1,2,3,4]

In [7]:
x,y,z,w = t

ValueError: not enough values to unpack (expected 5, got 4)

In [5]:
z

3

In [22]:
diagonal = ( ( (10-5)**2) + ( (10-5)**2) ) ** 0.5

In [23]:
diagonal

7.0710678118654755

In [37]:
7 < (8)

True